## R script for OMOP to load Care Site data

### Setup

In [78]:
library(DBI)

# make db connection
# for format of database file see db/env/db_conf.txt
db <- read.delim( '../../db/env/feta.txt', header=TRUE, sep=' ' )
con <- dbConnect(RPostgres::Postgres(),
                 dbname = toString(db$database[1]),
                 host = toString(db$host[1]),
                 port = 5432,
                 user = toString(db$user),
                 password = toString(db$pass))

# check the connection
dbListTables(con)

[1] "spatial_ref_sys"      "geography_columns"    "geometry_columns"    
 [4] "raster_columns"       "raster_overviews"     "hazard_point"        
 [7] "data_source"          "dd_type"              "dd_att_unit"         
[10] "dd_att_name"          "hz_att_category"      "locations"           
[13] "utm_grid"             "hz_att_unit"          "hz_att_name"         
[16] "hz_type"              "persons"              "care_site"           
[19] "dd_att_category"      "hz_attribute"         "location_history"    
[22] "site_history"         "dd_attribute"         "demographic_division"
[25] "sites"

## Load Site data

In [ ]:
sitedata <- read.csv('uhealth-locations_clean.csv',header=FALSE)

In [ ]:
for (i in 1:nrow(sitedata)) {
    
    # create location with geometry
    sql_createsite <- 'INSERT INTO sites ("site_id","latitude","longitude","geom") 
                       VALUES (DEFAULT,$1,$2,ST_SetSRID(ST_MakePoint($2,$1),4326))
                       RETURNING site_id'
    res <- dbSendQuery(con,sql_createsite)
    dbBind(res, list(sitedata[i,2],sitedata[i,3]))
    site_ID <- dbFetch(res)[1,1]
    dbClearResult(res)

    # create care site record
    sql_createcaresite <- 'insert into care_site ("care_site_id","care_site_name")
                       values(DEFAULT,$1)
                       returning care_site_id;'
    res <- dbSendQuery(con,sql_createcaresite)
    dbBind(res, list(sitedata[i,1]))
    care_site_ID <- dbFetch(res)[1,1]
    dbClearResult(res)
    
    # link care site and location
    sql_createsitehist = 'INSERT INTO site_history ("site_id","entity_id")
                          VALUES ($1,$2);'
    res <- dbSendQuery(con,sql_createsitehist)
    dbBind(res, list(site_ID,care_site_ID))
    dbClearResult(res)

    print (paste(sitedata[i,1],sitedata[i,2],sitedata[i,3]))

}